In [1]:
!python -m pip install langchain langchain-core langchain-community langchain-experimental --quiet
!python -m pip install -U langchain-google-genai --quiet
%pip install --upgrade --quiet  yfinance
%pip install langchain wikipedia-api --quiet
%pip install wikipedia --quiet
%pip install mlflow --quiet


[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
from google import genai
from google.genai.types import HttpOptions
from langchain_community.tools.yahoo_finance_news import YahooFinanceNewsTool
from langchain_core.prompts import ChatPromptTemplate
from langchain.chat_models import init_chat_model
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
from pydantic import BaseModel, Field
from typing import Dict, Any

from langchain_core.output_parsers import StrOutputParser,JsonOutputParser

parser = StrOutputParser()
parser_1=JsonOutputParser()

###Intialising the client
client = genai.Client(http_options=HttpOptions(api_version='v1'))
from pydantic import BaseModel, Field
import pprint

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [3]:
model_name='gemini-2.0-flash'
model = init_chat_model(model_name, model_provider="google_genai")

# Load API keys from .env file
from dotenv import load_dotenv

load_dotenv(override=True)

True

In [4]:
import mlflow
mlflow.set_tracking_uri("http://20.75.92.162:5000")

mlflow.set_experiment("MP_Sentiment_Analysis")

company_name=input("Enter the company name:")
print(company_name)

Tesla Inc


In [5]:
stock_tickers = {
    "Apple Inc": "AAPL",
    "Microsoft Corporation": "MSFT",
    "Amazon.com Inc": "AMZN",
    "Alphabet Inc": "GOOGL",
    "NVIDIA Corporation": "NVDA",
    "Meta Platforms Inc": "META",
    "Broadcom Inc": "AVGO",
    "Tesla Inc": "TSLA",
    "Taiwan Semiconductor": "TSM",
    "Berkshire Hathaway Inc": "BRK.B",
    "JPMorgan Chase & Co": "JPM",
    "Walmart Inc": "WMT",
    "Oracle Corporation": "ORCL",
    "Eli Lilly and Company": "LLY",
    "Visa Inc": "V",
    "Mastercard Inc": "MA",
    "Netflix Inc": "NFLX",
    "Exxon Mobil Corporation": "XOM",
    "Costco Wholesale Corporation": "COST",
    "Johnson & Johnson": "JNJ",
    "The Home Depot Inc": "HD",
    "Palantir Technologies Inc": "PLTR",
    "AbbVie Inc": "ABBV",
    "Bank of America Corporation": "BAC",
    "Procter & Gamble": "PG",
    "UnitedHealth Group": "UNH",
    "Chevron Corporation": "CVX",
    "ASML Holding N.V.": "ASML",
    "SAP SE": "SAP",
    "General Electric Company": "GE",
    "The Coca-Cola Company": "KO",
    "T-Mobile US Inc": "TMUS",
    "Cisco Systems Inc": "CSCO",
    "Wells Fargo & Company": "WFC",
    "Philip Morris International": "PM",
    "Advanced Micro Devices Inc": "AMD",
    "Toyota Motor Corporation": "TM",
    "Morgan Stanley": "MS",
    "AstraZeneca PLC": "AZN",
    "The Goldman Sachs Group Inc": "GS"
}

In [7]:
### A function to extract the stock ticker or symbol
@mlflow.trace
def get_stock_ticker(company_name):
    lookup = stock_tickers
    return lookup.get(company_name, "NOT AVAILABLE")
stock_code = get_stock_ticker(company_name)
print(stock_code)

TSLA


Trace(trace_id=tr-a232a818d4aaedd8e4c285e8ad7c6b78)

In [8]:
class StockNewsTool(BaseModel):
    """Input to the Stock News tool."""

    query: str = Field(
        description="Query to look up in Yahoo Finance News, should be stock market listed tickers"
    )
@mlflow.trace
def company_news_func(stock_code_1:str):
    Company_news_tool = YahooFinanceNewsTool(

    name="Stock Sentiment Analyzer",
    description="Look up the latest stock news of a company in Yahoo finance news using the extracted stock symbol. Extract a concise list of recent headlines, article summaries as well as historical stock data if available.",
    args_schema=StockNewsTool,
    metadata={"use_case": "financial_analysis", "source": "YahooFinance"},
   
    return_direct=True,
    
    top_k= 5,
    verbose=True

)
    return Company_news_tool.run(stock_code_1)

# print(tool.run("AAPL"))
response=company_news_func(stock_code)

pprint.pprint(response)

'I Don't Have Hundreds Of Thousands Of Dollars,' Says Elon Musk's 21-Year-Old Daughter Who Lives With Roommates To Save On Rent
Vivian Wilson, daughter of Tesla Inc. (NASDAQ:TSLA) CEO Elon Musk, said she lives with three roommates in Los Angeles because it is "cheaper" and she often worries about money. "People assume I have a lot of money. I don't have hundreds of thousands of dollars at my disposal," the 21-year-old told New York magazine recently. Despite being the oldest of Musk's 14 children, Wilson said she does not benefit financially from his fortune and is working to build her own path. Don't Miss: Kevin O'Leary("'I Don't Have Hundreds Of Thousands Of Dollars,' Says Elon Musk's "
 '21-Year-Old Daughter Who Lives With Roommates To Save On Rent\n'
 'Vivian Wilson, daughter of Tesla Inc. (NASDAQ:TSLA) CEO Elon Musk, said she '
 'lives with three roommates in Los Angeles because it is "cheaper" and she '
 'often worries about money. "People assume I have a lot of money. I don\'t '

Trace(trace_id=tr-a30281261b27b2eec023172cf8d00e00)

In [9]:
from langchain.chains import LLMChain
mlflow.gemini.autolog()

In [10]:
import mlflow

# Use double curly braces for variables in the template
initial_template = """
You are a financial analyst. Given the following news about {company} {ticker}, analyze the sentiment and extract structured insights.

News:
{news}

Return a dictionary with the following fields:
{{
 "company_name": "{company}",
 "stock_code": "{ticker}",
 "newsdesc": "...",
 "sentiment": "Positive/Negative/Neutral",
 "people_names": [],
 "places_names": [],
 "other_companies_referred": [],
 "related_industries": [],
 "market_implications": "",
 "confidence_score": 0.0
}}
"""


# Register a new prompt
prompt = mlflow.genai.register_prompt(
    name="MP-SentimentAnalysis-prompt",
    template=initial_template,
    # Optional: Provide a commit message to describe the changes
    commit_message="Initial commit",
    # Optional: Set tags applies to the prompt (across versions)
    tags={
        "author": "Mainak Paul",
        "task": "Sentiment analysis",
        "language": "en",
        'llm': 'gemini-2.0-flash'
    },
)

# The prompt object contains information about the registered prompt
print(f"Created prompt '{prompt.name}' (version {prompt.version})")

2025/09/25 15:56:57 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for prompt version to finish creation. Prompt name: MP-SentimentAnalysis-prompt, version 1


Created prompt 'MP-SentimentAnalysis-prompt' (version 1)


In [12]:
sentiment_prompt = mlflow.genai.load_prompt("prompts:/MP-SentimentAnalysis-prompt/1")
print(sentiment_prompt.template)


You are a financial analyst. Given the following news about {company} {ticker}, analyze the sentiment and extract structured insights.

News:
{news}

Return a dictionary with the following fields:
{{
 "company_name": "{company}",
 "stock_code": "{ticker}",
 "newsdesc": "...",
 "sentiment": "Positive/Negative/Neutral",
 "people_names": [],
 "places_names": [],
 "other_companies_referred": [],
 "related_industries": [],
 "market_implications": "",
 "confidence_score": 0.0
}}



In [13]:
@mlflow.trace
def run_sentiment_analysis(model, sentiment_prompt, company_name, stock_code, news_text):
    """
    Run sentiment analysis on news related to a company using an LLM chain.

    Args:
        model: The language model instance (e.g., from OpenAI, HuggingFace, etc.).
        sentiment_prompt: A PromptTemplate object or template string for sentiment analysis.
        company_name (str): The name of the company.
        stock_code (str): The ticker symbol of the company.
        news_text (str): The news article or snippet for analysis.

    Returns:
        str: The model's response (e.g., sentiment analysis result).
    """
    
    # Create the PromptTemplate object if it's not already
    if isinstance(sentiment_prompt, str):
        prompt_1 = PromptTemplate(
            input_variables=["company", "ticker", "news"],
            template=sentiment_prompt
        )
    else:
        prompt_1 = sentiment_prompt

    # Create the LLM chain and run it
    chain = LLMChain(llm=model, prompt=prompt_1)
    response_final = chain.run(company=company_name, ticker=stock_code, news=news_text)
    response_final=parser_1.invoke(response_final)
    
    return response_final

In [14]:
final_response = run_sentiment_analysis(
    model=model,
    sentiment_prompt=sentiment_prompt.template,
    company_name=company_name,
    stock_code=stock_code,
    news_text=response
)

final_response

/tmp/ipykernel_3005/2600143604.py:27: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=model, prompt=prompt_1)
/tmp/ipykernel_3005/2600143604.py:28: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response_final = chain.run(company=company_name, ticker=stock_code, news=news_text)


{'company_name': 'Tesla Inc',
 'stock_code': 'TSLA',
 'newsdesc': "Elon Musk's daughter, Vivian Wilson, states she lives with roommates to save on rent and doesn't have access to her father's wealth, despite being his daughter. She is working to build her own path.",
 'sentiment': 'Neutral',
 'people_names': ['Elon Musk', 'Vivian Wilson', "Kevin O'Leary"],
 'places_names': ['Los Angeles', 'New York'],
 'other_companies_referred': [],
 'related_industries': ['Automotive'],
 'market_implications': "This news is unlikely to have a significant direct impact on Tesla's stock price. While it involves Elon Musk, it's a personal story about his daughter's financial independence and doesn't directly affect Tesla's operations, financials, or competitive position.",
 'confidence_score': 0.8}

Trace(trace_id=tr-1aeebee0929ec6eeb663baaacc71449a)